# Reviewing Bag of Words with Keras

In this document, we use a few simple techniques to try and complete the task set out by the Kaggle StumbleUpon Competition listed below.

https://www.kaggle.com/c/stumbleupon

**Competition**: Some web pages, such as news articles or seasonal recipes, are only relevant for a short period of time. Others continue to be important for a long time.

**Goal**: The goal is to identify pages which pages will be relevant for a short span of time, and which will be relevant for a long span on time and are thus considered "evergreen".

**Evaluation**: Area under the curve (AUC)

## 1. Initial Setup

### 1.1 Import Python Packages

In [1]:
# quick hack to fix import path
# import sys; sys.path.append('/Users/julianalverio/code/conda/envs/sac/lib/python3.6/site-packages/')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf

# data manipulation
import pandas as pd
import numpy as np

# plots
%matplotlib inline
import random
import matplotlib
import matplotlib.pyplot as plt
import pylab as pl

# classification algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

# dimensionality reduction
from sklearn.decomposition import PCA

# cross-validation
from sklearn.model_selection import train_test_split
from sklearn import model_selection

# text features
import re
from nltk import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# model evaluation
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings("ignore")

import os
os.chdir(os.getcwd())

/home/ubuntu/conda/envs/conda_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/conda/envs/conda_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/conda/envs/conda_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/conda/envs/conda_env/lib/python3.6/s

Here we look at the data.

In [2]:
# Read stumbleupon data using pandas
data = pd.read_table("train.tsv", sep= "\t")

# Look at data
df = pd.DataFrame(data["boilerplate"]) # extract data
df # print out dataframe to look at it

,boilerplate
0,"{""title"":""IBM Sees Holographic Calls Air Breat..."
1,"{""title"":""The Fully Electronic Futuristic Star..."
2,"{""title"":""Fruits that Fight the Flu fruits tha..."
3,"{""title"":""10 Foolproof Tips for Better Sleep ""..."
4,"{""title"":""The 50 Coolest Jerseys You Didn t Kn..."
5,"{""url"":""conveniencemedical genital herpes home..."
6,"{""title"":""fashion lane American Wild Child "",""..."
7,"{""url"":""insidershealth article racing for reco..."
8,"{""title"":""Valet The Handbook 31 Days 31 days"",..."
9,"{""url"":""howsweeteats 2010 03 24 cookies and cr..."


### 1.2 Using Numerical Features (same as last week)

In [3]:
# Alchemy category, converting to one-hots
df = data['alchemy_category']   # 2K ? values
one_hots = pd.get_dummies(data['alchemy_category'])
df = one_hots
rename_dict = {'?': 'alchemy_cat_?'}
df = df.rename(columns=rename_dict)

# FrameTagRatio, leaving as continuous number
df_var = data['frameTagRatio']
df['frame_tag_ratio'] = df_var

# link word score, 0-100 gaussian, keeping continuous
df['link_word_score'] = data['linkwordscore']

# alchemy category score, with replacing missing values with random
df_var = data['alchemy_category_score']
df_var_temp = df_var.apply(lambda x: np.random.random() if x == '?' else float(x)).astype('float32')
df['alchemy_category_score'] = df_var_temp

# num word in url -- discrete 0-25 to custom binning from looking at the histogram
df_var = data['numwords_in_url']
bins = [0, 6, 8, 13, 25]
df_var_temp = pd.cut(x=df_var, bins=bins, right=True, labels=['num_words_url_bin_0', 'num_words_url_bin_1', 'num_words_url_bin_2', 'num_words_url_bin_3'])
dummies = pd.get_dummies(df_var_temp)
df = pd.concat([df, dummies], axis=1)

# parameterized_link_ratio -- leaving as continuous, right-half gaussian
df['parameterized_link_ratio'] = data['parametrizedLinkRatio']

# spelling errors ratio -- leaving as continuous
df['spelling_errors_ratio'] = data['spelling_errors_ratio']

# embed_ratio -- bimodal continuous binned into 2 bins
df_var = pd.DataFrame(data['embed_ratio'])
df_var = df_var['embed_ratio'].apply(lambda x: 1 if x > -1 else 0)
dummies = pd.get_dummies(df_var)
rename = {0: 'embed_ratio_0', 1: 'embed_ratio_1'}
dummies = dummies.rename(columns=rename)
df = pd.concat([df, dummies], axis=1)

# html_ratio -- leaving continuous
df['html_ratio'] = data['html_ratio']

# lengthy_link_domain
df_var = pd.get_dummies(data['lengthyLinkDomain'])
rename = {0: 'lengthy_link_domain_0', 1: 'lengthy_link_domain_1'}
df_var = df_var.rename(columns=rename)
df = pd.concat([df, df_var], axis=1)

df['labels'] = data['label']

### 1.3 Creating Training and Testing Data Splits

In [4]:
# Split data into training and testing
train, val = train_test_split(df, test_size=0.5, train_size=0.5, random_state=234)

# Split testing into validation and test
val, test = train_test_split(val, test_size=0.5, train_size=0.5, random_state=675)

# Get labels for training dataset
train_labels = train['labels']
train = train.drop(['labels'], axis=1, inplace=False)

# Get labels for validation dataset
val_labels = val['labels']
val = val.drop(['labels'], axis=1, inplace=False)

# Get labels for testing dataset
test_labels = test['labels']
test = test.drop(['labels'], axis=1, inplace=False)

## 2. Bag of Words

### 2.1 Import Data

The data for this exercise has been placed within the folder that this notebook is in. Therefore, we can simply reference it below.

In [5]:
# Read stumbleupon data using pandas
data = pd.read_table("train.tsv", sep= "\t")

### 2.2 Using Count Vectorizer

Below is the code that we will re-run from last week. We have four major features which are relevant! I have written the descriptions from the function documentation for your convenience. Because of this, know that it is often your responsibility to do this step. Understanding what functions you are using solely relies on how diligently you reference the documentation! The link to the documentation is here, but please do this yourself for the exercise. Here is the [link](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer.fit) to the `CountVectorizer` class documentation.

- *min_df* = minimum frequencey cut-off
    - min_dffloat in range [0.0, 1.0] or int, default=1: When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.
- *max_features* = take the top 1000 most common feature
    - max_featuresint or None, default=None: If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.
- *strip_accents* = to handle non english letters
    - strip_accents{‘ascii’, ‘unicode’, None}: Remove accents and perform other character normalization during the preprocessing step. ‘ascii’ is a fast method that only works on characters that have an direct ASCII mapping. ‘unicode’ is a slightly slower method that works on any characters. None (default) does nothing. Both ‘ascii’ and ‘unicode’ use NFKD normalization from unicodedata.normalize.
- *ngram_range* = we are doing bag of word features here
    - ngram_rangetuple (min_n, max_n), default=(1, 1): The lower and upper boundary of the range of n-values for different word n-grams or char n-grams to be extracted. All values of n such such that min_n <= n <= max_n will be used. For example an ngram_range of (1, 1) means only unigrams, (1, 2) means unigrams and bigrams, and (2, 2) means only bigrams. Only applies if analyzer is not callable.

#### 2.2.1 Instantiate Class

In this first function call, we instantiate an instance of the class CountVectorizer.

In [6]:
# Instantiate our class
unigram_dtm = CountVectorizer(min_df= 10,  max_features= 1000, strip_accents= "unicode",
                          ngram_range=(1, 1), binary = True)
print(unigram_dtm) # by printing this variable, we see that it outputs a class description

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=1000, min_df=10,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents='unicode', token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)


#### 2.2.2 Make Training and Testing Sets to do Bag of Words

We have talked about generating training and testing data. Below are functions which do this. If the code is confusing, reference the documentation associated with the functions! Here is an example which shows you what this function does. When writing your own code, try and look at examples or make examples like this to mess around with your code.

In [7]:
# Example of using the function
X = np.arange(10).reshape((5, 2))
print("Our data:\n{}".format(X))

X_train, X_test = train_test_split(X, test_size=0.5, random_state=88)
print("Our training set:\n{}".format(X_train))
print("Our testing set:\n{}".format(X_test))

Our data:
[[0 1]
 [2 3]
 [4 5]
 [6 7]
 [8 9]]
Our training set:
[[8 9]
 [0 1]]
Our testing set:
[[4 5]
 [6 7]
 [2 3]]


Here is where we split the data relevant with our problem.

In [8]:
# Split Data Before Generating Bag of Words Representation
train_boilerplate, val_boilerplate = train_test_split(data['boilerplate'], test_size=0.5, train_size=0.5, random_state=234)
val_boilerplate, test_boilerplate = train_test_split(val_boilerplate, test_size=0.5, train_size=0.5, random_state=675)

print(type(train_boilerplate)) # when looking at the output, keep in mind that this data type is a SPARSE matrix
print(train_boilerplate.shape)

<class 'pandas.core.series.Series'>
(3697,)


Following the instantiation of the class, we then run some functions which help us to build a Bag of Words representation of the data.

#### 2.2.3 Learn Vocabulary from Document

Below, we call the `fit()` function in order to learn a vocabulary from one or more documents. 

#### 2.2.4 Generate Bag of Words Vectors from Document

Following this, we call the `transform()` function on one or more documents as needed to encode each as a vector. 

In this case, we use the same data for each function. Think about why this is!

Here, we actually implement the functions for our data.

In [9]:
# Make Bag of Words Representation
unigram_dtm.fit(train_boilerplate) # here we are creating a dictionary
train_text = unigram_dtm.transform(train_boilerplate) # here, we encode this document as a vector in our dictionary space
val_text = unigram_dtm.transform(val_boilerplate) # why do we have to transform but not fit?

# Look at some of the variables
print(type(train_text)) # when looking at the output, keep in mind that this data type is a SPARSE matrix
print(train_text) # looking at the data, what may sparse mean?

<class 'scipy.sparse.csr.csr_matrix'>
  (0, 0)	1
  (0, 8)	1
  (0, 9)	1
  (0, 10)	1
  (0, 14)	1
  (0, 24)	1
  (0, 25)	1
  (0, 34)	1
  (0, 38)	1
  (0, 41)	1
  (0, 44)	1
  (0, 53)	1
  (0, 55)	1
  (0, 57)	1
  (0, 60)	1
  (0, 63)	1
  (0, 71)	1
  (0, 72)	1
  (0, 73)	1
  (0, 77)	1
  (0, 78)	1
  (0, 81)	1
  (0, 85)	1
  (0, 89)	1
  (0, 90)	1
  :	:
  (3696, 918)	1
  (3696, 919)	1
  (3696, 931)	1
  (3696, 941)	1
  (3696, 942)	1
  (3696, 945)	1
  (3696, 947)	1
  (3696, 952)	1
  (3696, 954)	1
  (3696, 956)	1
  (3696, 959)	1
  (3696, 962)	1
  (3696, 966)	1
  (3696, 967)	1
  (3696, 968)	1
  (3696, 970)	1
  (3696, 973)	1
  (3696, 975)	1
  (3696, 977)	1
  (3696, 980)	1
  (3696, 984)	1
  (3696, 989)	1
  (3696, 991)	1
  (3696, 992)	1
  (3696, 994)	1


#### 2.2.5 Explore Your Output

In [10]:
# Randomly choose features
np.random.choice(unigram_dtm.get_feature_names(), 10)

array(['since', '21', 'rack', 'sleep', 'cooking', 'body', 'might',
       'smooth', 'cheese', 'these'], dtype='<U12')

In [11]:
xx = train_text.toarray() # convert data type to something easier to look at
print(xx.shape) # look at shape of array
print("\nLooking at our training text matrix:\n{}".format(xx)) # look at data

(3697, 1000)

Looking at our training text matrix:
[[1 0 0 ... 0 1 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 1 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 [1 0 1 ... 0 0 0]]


In [12]:
# Print dimensionality of training data
print(train.shape)
# Convert to array and print dimensions
print(train_text.toarray().shape)

(3697, 28)
(3697, 1000)


#### 2.2.6 Add New Features to Training Set

In [13]:
# Create training and validation datasets with text with concatenation
train_with_text = pd.concat([train.reset_index(drop = True), pd.DataFrame(train_text.toarray())], axis=1)
val_with_text = pd.concat([val.reset_index(drop = True), pd.DataFrame(val_text.toarray())], axis=1)
train_with_text.head()

# print(type(train_with_text))
# print(type(val_with_text))

,alchemy_cat_?,arts_entertainment,business,computer_internet,culture_politics,gaming,health,law_crime,recreation,religion,...,990,991,992,993,994,995,996,997,998,999
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,1,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


#### 2.2.7 Do Logistic Regression with new Feature Set

In [14]:
# Create logistic regression model with sklearn
model = LogisticRegression()

# Fit, generate predictions, and evaluate model
model.fit(train_with_text, train_labels.values)
preds = model.predict_proba(val_with_text)[:,1]
score = roc_auc_score(val_labels, preds)
print("Score is {}".format(score))

Score is 0.7780866732657976


### 2.3 Keras Implementation

#### 2.3.1 Instantiate Class

In [15]:
from keras.preprocessing.text import Tokenizer
# define 5 documents
docs = ['Well done!',
        'Good work',
        'Great effort',
        'nice work',
        'Excellent!']
# create the tokenizer
t = Tokenizer()
# fit the tokenizer on the documents
t.fit_on_texts(docs)
# summarize what was learned
print(t.word_counts)
# print(t.document_count)
# print(t.word_index)
# print(t.word_docs)
# integer encode documents
encoded_docs = t.texts_to_matrix(docs, mode='count')
print(encoded_docs)

OrderedDict([('well', 1), ('done', 1), ('good', 1), ('work', 2), ('great', 1), ('effort', 1), ('nice', 1), ('excellent', 1)])
[[0. 0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]]


Using TensorFlow backend.


In [16]:
from keras.preprocessing.text import Tokenizer

# create the tokenizer
num_words = 10000
t = Tokenizer(num_words = num_words)
# unigram_dtm = CountVectorizer(min_df= 10,  max_features= 1000, strip_accents= "unicode", ngram_range=(1, 1))

#### 2.3.2 Learn Vocabulary from Document

In [17]:
# fit the tokenizer on the documents
t.fit_on_texts(train_boilerplate)

#### 2.3.3 Generate Bag of Words Vectors from Document

In [18]:
from scipy import sparse

train_text_k = t.texts_to_matrix(train_boilerplate, mode='binary')
print(train_text_k)
train_text_k = sparse.csr_matrix(train_text_k)
print(train_text_k)

val_text_k = t.texts_to_matrix(val_boilerplate, mode='binary')
val_text_k = sparse.csr_matrix(val_text_k)

[[0. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]]
  (0, 1)	1.0
  (0, 2)	1.0
  (0, 3)	1.0
  (0, 4)	1.0
  (0, 5)	1.0
  (0, 6)	1.0
  (0, 7)	1.0
  (0, 8)	1.0
  (0, 9)	1.0
  (0, 11)	1.0
  (0, 12)	1.0
  (0, 13)	1.0
  (0, 14)	1.0
  (0, 15)	1.0
  (0, 16)	1.0
  (0, 17)	1.0
  (0, 18)	1.0
  (0, 19)	1.0
  (0, 20)	1.0
  (0, 21)	1.0
  (0, 22)	1.0
  (0, 23)	1.0
  (0, 24)	1.0
  (0, 25)	1.0
  (0, 26)	1.0
  :	:
  (3696, 6727)	1.0
  (3696, 6940)	1.0
  (3696, 6955)	1.0
  (3696, 6969)	1.0
  (3696, 7005)	1.0
  (3696, 7038)	1.0
  (3696, 7371)	1.0
  (3696, 7458)	1.0
  (3696, 7547)	1.0
  (3696, 7706)	1.0
  (3696, 8010)	1.0
  (3696, 8169)	1.0
  (3696, 8219)	1.0
  (3696, 8234)	1.0
  (3696, 8266)	1.0
  (3696, 8398)	1.0
  (3696, 8590)	1.0
  (3696, 8593)	1.0
  (3696, 9088)	1.0
  (3696, 9097)	1.0
  (3696, 9404)	1.0
  (3696, 9607)	1.0
  (3696, 9612)	1.0
  (3696, 9665)	1.0
  (3696, 9711)	1.0


#### 2.3.4 Explore Your Output

In [19]:
print(t.word_counts) # ordered dictionary
# print(t.document_count) # number of documents
# print(t.word_index) # index of where the word is
# print(t.word_docs) # look at 

OrderedDict([('title', 3836), ('technology', 767), ('world', 1540), ('s', 11840), ('news', 1283), ('updates', 85), ('of', 34646), ('august', 241), ('2011', 1569), ('body', 4928), ('things', 944), ('are', 8021), ('changing', 64), ('rapidly', 35), ('just', 3698), ('over', 2641), ('a', 39681), ('decade', 74), ('ago', 356), ('geek', 56), ('was', 5207), ('used', 1269), ('as', 7583), ('an', 4470), ('insult', 8), ('it', 16760), ('sometimes', 313), ('still', 1032), ('is', 15563), ('but', 5935), ('usually', 360), ('by', 5780), ('someone', 305), ('no', 2212), ('importance', 36), ('and', 45180), ('huge', 201), ('chip', 287), ('on', 11448), ('his', 1994), ('shoulder', 111), ('the', 74050), ('word', 156), ('now', 1800), ('associated', 155), ('with', 14458), ('mark', 181), ('zuckerberg', 2), ('page', 302), ('brin', 3), ('or', 8673), ('perhaps', 171), ('bill', 143), ('gates', 19), ('simple', 745), ('logic', 22), ('to', 39566), ('deduce', 1), ('that', 13910), ('evolution', 32), ('mankind', 10), ('will

#### 2.3.5 Add New Features to Training Set

In [20]:
# Create training and validation datasets with text with concatenation
train_with_text_k = pd.concat([train.reset_index(drop = True), pd.DataFrame(train_text_k.toarray())], axis=1)
val_with_text_k = pd.concat([val.reset_index(drop = True), pd.DataFrame(val_text_k.toarray())], axis=1)
train_with_text_k.head()

,alchemy_cat_?,arts_entertainment,business,computer_internet,culture_politics,gaming,health,law_crime,recreation,religion,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,1,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### 2.3.6 Do Logistic Regression with New Feature Set in Keras

In [21]:
# Create logistic regression model with sklearn
model = LogisticRegression()

# Fit, generate predictions, and evaluate model
print(type(train_with_text))
print(type(val_with_text))
model.fit(train_with_text_k, train_labels.values)
preds = model.predict_proba(val_with_text_k)[:,1]
score = roc_auc_score(val_labels, preds)
print("Score is {}".format(score))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
Score is 0.8097519709638241


In [22]:
# Create logistic regression model with Keras
from keras.models import Sequential
from keras.layers import Dense

model_k = Sequential() # instantiate class

# Build Neural Network
num_words = 10000 + 28
model_k.add(Dense(2, input_dim=num_words, activation='relu')) # input (input is just original data) and hidden layer
model_k.add(Dense(1, activation='sigmoid')) # output layer

# Compile
model_k.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) # compiling into TensorFlow

# Fit
model_k.fit(train_with_text_k, train_labels.values, epochs=5, batch_size=100)

# Evaluate the keras model
_, accuracy = model_k.evaluate(val_with_text_k, val_labels.values)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/5
3697/3697 [==============================] - 1s 373us/step - loss: 0.5503 - acc: 0.7149
Epoch 2/5
3697/3697 [==============================] - 1s 135us/step - loss: 0.4358 - acc: 0.8020
Epoch 3/5
3697/3697 [==============================] - 0s 134us/step - loss: 0.3896 - acc: 0.8456
Epoch 4/5
3697/3697 [==============================] - 0s 135us/step - loss: 0.3526 - acc: 0.8615
Epoch 5/5
1849/1849 [==============================] - 0s 94us/step
Accuracy: 79.66
